# Known Cancer Predisposition Genes, without autosomal recessive genes for Carcinoma
----- 
- Date : 17th May 2019
- using R3.5 kernal
- on 89

In [1]:
source("~/bin/da.R")

In [2]:
suppressPackageStartupMessages({
    library(tidyverse) 
    library(vcd)
    library(readr)
    library(maftools)
    library(corrplot)
    library(knitr)
    library(wordcloud)
    library(RColorBrewer)})

In [3]:
packageVersion("maftools")

[1] ‘1.8.0’

## Mutationa Data

In [4]:
file_name="../KCPG_Variants/data/Mutation_data_cleaned_051019.xls"

In [5]:
df= readxl::read_excel(file_name)

In [6]:
df %>% dim

[1] 3016   84

In [7]:
knitr::kable( sort( table(df$Tumor_types ),decreasing = T), format="pandoc")



Var1     Freq
------  -----
CNS       613
WLM       405
NBL       343
RHB       259
OS        241
HGG       229
RB        221
EWS       192
STS       177
GCT       139
ACT        79
LGG        52
CA         35
OST        18
RCC         5
GICT        2
LMPRT       2
PGL         2
BCC         1
NM          1

In [8]:
cbind( df$Known_Cancer_Predisposition_Genes %>% table )

known,290
suggesting,2726


In [9]:
df = df[df$Known_Cancer_Predisposition_Genes == 'known',]
df =droplevels(df)

In [10]:
df%>% dim

[1] 290  84

In [11]:
df$Mode_of_Inheritance_abb %>% table

.
   AD    AR AR/AD   XLD 
  153    92    43     2 

In [12]:
table(df$Mode_of_Inheritance_abb , df$genotype)

       
        Het Hom
  AD    153   0
  AR     92   0
  AR/AD  43   0
  XLD     1   1

## Remove AR

In [13]:
df = df[df$Mode_of_Inheritance_abb != 'AR',]
df = droplevels(df)

In [14]:
df %>% dim

[1] 198  84

In [15]:
# sanity check
df$Mode_of_Inheritance_abb %>% table

.
   AD AR/AD   XLD 
  153    43     2 

In [16]:
len( unique(df$Tumor_Sample_Barcode) )

[1] 182

In [17]:
len( unique(df$Hugo_Symbol) )

[1] 54

## EDA Clinical_Data

In [18]:
file_name_clinical = "../KCPG_Variants/data/Combined_Clinical_Data_03.xlsx"

In [19]:
df2 = readxl::read_excel( file_name_clinical )

In [20]:
colnames(df2)

[1] "Tumor_Sample_Barcode" "Tumor_type"           "Tumor_type2"         
 [4] "Age_of_Onset"         "Sex"                  "Age_group"           
 [7] "sample_type"          "sequencing_type"      "Access_unit"         
[10] "Status"               "Ethnicity"            "Race"

In [21]:
table(df2$Tumor_type , df2$Access_unit )

                                      
                                       CCF PCGP SJLIFE
  Adrenocortical Carcinoma               1   19      2
  Basal Cell Carcinoma                   0    0      1
  Carcinoma                              0    0     14
  Central Nervous System                 1    0    322
  Ewing's Sarcoma                        6    0     89
  Germ Cell Tumor                        1    0     73
  Giant Cell Tumor                       0    0      3
  High Grade Glioma                      5   75      0
  Low Grade Glioma                       2   15      7
  Low Malignant Potential Renal Tumors   2    0      0
  Neuroblastoma                          3   48    139
  Non-Malignant Tumor                    1    0      3
  Osteosarcoma                          14    2    113
  Other Solid Tumor                      0    0     10
  Paraganglioma                          0    0      1
  Renal Cell Carcinoma                   1    0      2
  Retinoblastoma          

In [22]:
fct_count( df2$Tumor_type , sort = T)

f,n
Central Nervous System,323
Wilms' tumor,207
Neuroblastoma,190
Rhabdomyosarcoma,134
Osteosarcoma,129
Retinoblastoma,98
Ewing's Sarcoma,95
Soft Tissue Sarcoma,93
High Grade Glioma,80
Germ Cell Tumor,74


In [23]:
## sanity check
table(df2$Tumor_type , df2$Tumor_type2 )

                                      
                                       ACT BCC  CA CNS EWS GCT GICT HGG LGG
  Adrenocortical Carcinoma              22   0   0   0   0   0    0   0   0
  Basal Cell Carcinoma                   0   1   0   0   0   0    0   0   0
  Carcinoma                              0   0  14   0   0   0    0   0   0
  Central Nervous System                 0   0   0 323   0   0    0   0   0
  Ewing's Sarcoma                        0   0   0   0  95   0    0   0   0
  Germ Cell Tumor                        0   0   0   0   0  74    0   0   0
  Giant Cell Tumor                       0   0   0   0   0   0    3   0   0
  High Grade Glioma                      0   0   0   0   0   0    0  80   0
  Low Grade Glioma                       0   0   0   0   0   0    0   0  24
  Low Malignant Potential Renal Tumors   0   0   0   0   0   0    0   0   0
  Neuroblastoma                          0   0   0   0   0   0    0   0   0
  Non-Malignant Tumor                    0   0   

In [24]:
## sanity check
df %>% dim
df2 %>% dim

[1] 198  84

[1] 1507   12

In [25]:
ttype ="Carcinoma"

In [26]:
df2_ss = df2[df2$Tumor_type == ttype,]
ss =df$Tumor_Sample_Barcode %in% intersect( df$Tumor_Sample_Barcode, unique( df2_ss$Tumor_Sample_Barcode ))
df_ss = df[ ss, ]
df_ss = droplevels(df_ss)
df2_ss = droplevels(df2_ss)

In [27]:
df_ss$Hugo_Symbol=as.character(df_ss$Hugo_Symbol) 

In [28]:
table(df2_ss$Tumor_type , df2_ss$Tumor_type2 )

           
            CA
  Carcinoma 14

In [29]:
setdiff( df_ss$Tumor_Sample_Barcode, df2_ss$Tumor_Sample_Barcode)

character(0)

In [34]:
cbind(dim(df_ss), dim(df2_ss))

0,14
84,12


In [40]:
df2_ss$Tumor_Sample_Barcode

[1] "SJCA019119" "SJCA019120" "SJCA041980" "SJCA042494" "SJCA019116"
 [6] "SJCA017893" "SJCA042488" "SJCA019117" "SJCA042484" "SJCA041905"
[11] "SJCA018563" "SJCA019767" "SJCA041981" "SJCA019193"

In [45]:
intersect( df$Tumor_Sample_Barcode, df2_ss$Tumor_Sample_Barcode)

character(0)

In [47]:
df2_ss

Tumor_Sample_Barcode,Tumor_type,Tumor_type2,Age_of_Onset,Sex,Age_group,sample_type,sequencing_type,Access_unit,Status,Ethnicity,Race
SJCA019119,Carcinoma,CA,10.94,Male,Children,germline,WES,SJLIFE,live,"Non Spanish speaking, Non Hispanic",NA
SJCA019120,Carcinoma,CA,12.30,Female,Children,germline,WES,SJLIFE,live,"Non Spanish speaking, Non Hispanic",NA
SJCA041980,Carcinoma,CA,12.77,Female,Children,germline,WES,SJLIFE,live,"Non Spanish speaking, Non Hispanic",NA
SJCA042494,Carcinoma,CA,13.11,Female,Children,germline,WES,SJLIFE,live,"Non Spanish speaking, Non Hispanic",NA
SJCA019116,Carcinoma,CA,13.28,Male,Children,germline,WES,SJLIFE,live,"Non Spanish speaking, Non Hispanic",NA
SJCA017893,Carcinoma,CA,13.64,Male,Children,germline,WES,SJLIFE,live,"Non Spanish speaking, Non Hispanic",NA
SJCA042488,Carcinoma,CA,13.84,Female,Children,germline,WES,SJLIFE,live,"Non Spanish speaking, Non Hispanic",NA
SJCA019117,Carcinoma,CA,15.34,Female,Adolescent,germline,WES,SJLIFE,live,"Non Spanish speaking, Non Hispanic",NA
SJCA042484,Carcinoma,CA,15.51,Female,Adolescent,germline,WES,SJLIFE,live,"Non Spanish speaking, Non Hispanic",NA
SJCA041905,Carcinoma,CA,15.80,Female,Adolescent,germline,WES,SJLIFE,live,"Non Spanish speaking, Non Hispanic",NA


## Creating a MAF object

In [33]:
dfx = read.maf(maf=df_ss,clinicalData = df2_ss, isTCGA = FALSE, verbose = F, removeDuplicatedVariants = FALSE, useAll = TRUE )

ERROR: Error in read.maf(maf = df_ss, clinicalData = df2_ss): No non-synonymous mutations found
Check `vc_nonSyn`` argumet in `read.maf` for details


In [ ]:
dfx@summary

In [ ]:
dfx@gene.summary %>% head

# Number of unique genes after making MAF file

In [ ]:
dfx@gene.summary %>% dim

### Samples with top variants

In [ ]:
knitr::kable( dfx@variant.type.summary %>% head,format = "pandoc")

In [ ]:
knitr::kable( dfx@variants.per.sample %>% head,format = "pandoc")

### Number of Genes with 2 or more mutations

In [ ]:
gs = as.data.frame( dfx@gene.summary )  
gf_sx = nrow( gs[ gs$MutatedSamples >= 2,] )
gf_sx

In [ ]:
total_genes=length(gs$Hugo_Symbol)

In [ ]:
total_genes

In [ ]:
gs

In [ ]:
gs_mat = gs %>% dplyr::select(-c('Hugo_Symbol','total', 'AlteredSamples', 'MutatedSamples') )
rownames(gs_mat)=gs$Hugo_Symbol
options(repr.plot.height=4,repr.plot.width=5)
ggballoonplot(gs_mat)

In [ ]:
options(repr.plot.width=5, repr.plot.height=4)
gs %>% ggdotchart("Hugo_Symbol","MutatedSamples", 
                  rotate = TRUE,
                  sorting = "descending",
                  show.label = F,size = "MutatedSamples", 
                  ggtheme = theme_pubr() )

### plotmafSummary
to plot the summary of the maf file, which displays number of variants in each sample as a stacked barplot and variant types as a boxplot summarized by Variant_Classification.

In [ ]:
options(repr.plot.width=12, repr.plot.height=7)
plotmafSummary(maf=dfx, rmOutlier=F ,dashboard=TRUE,addStat='median', textSize=2,top=total_genes, showBarcodes=F )

In [ ]:
options(repr.plot.width=12, repr.plot.height=7)
out_svg = paste0('../KCPG_Variants/plots/KCP_wo_AR_','TT-',gsub("'","", gsub(' ','_',ttype)),'_plotmafsummary_01.svg')
paste0("Saving : ", out_svg)
svg(out_svg, height=7, width=12)
plotmafSummary(maf=dfx, rmOutlier=F ,dashboard=TRUE,addStat='median', textSize=2,top=total_genes, showBarcodes=F )
dev.off()

In [ ]:
options(repr.plot.width=9, repr.plot.height=5)
plotmafSummary(maf = dfx, rmOutlier = FALSE ,dashboard = F,addStat = 'median', textSize = 2, top = gf_sx, showBarcodes=F )

### Oncoplots or waterfall plots
- Adding Clinical Data on oncoplot

### Make Oncoplots with the total genes from the gene summary

In [ ]:
options(repr.plot.height=7,repr.plot.width=9)
oncoplot(maf = dfx,bgCol = "white",
        top = total_genes,
        
        fontSize = 10,
        SampleNamefontSize = 8,
        titleFontSize = 12,
        legendFontSize = 14,
        annotationTitleFontSize = 10,
        annotationFontSize = 9,
        
        drawColBar = TRUE,
        drawRowBar = TRUE,
        showTumorSampleBarcodes = F,
        sortByAnnotation = TRUE,
        writeMatrix = FALSE,
        clinicalFeatures = c("Age_group","Sex"))

In [ ]:
out_svg = paste0('../KCPG_Variants/plots/KCP_wo_AR_','TT-',gsub("'","", gsub(' ','_',ttype)),'_oncoplot_01.svg')
paste0("Saving : ", out_svg)
svg(out_svg, height=7, width=9)

oncoplot(maf = dfx,bgCol = "white",
        top = total_genes,
        
        fontSize = 10,
        SampleNamefontSize = 8,
        titleFontSize = 12,
        legendFontSize = 14,
        annotationTitleFontSize = 10,
        annotationFontSize = 9,
        
        drawColBar = TRUE,
        drawRowBar = TRUE,
        showTumorSampleBarcodes = F,
        sortByAnnotation = TRUE,
        writeMatrix = FALSE,
        clinicalFeatures = c("Age_group","Sex"))
dev.off()

In [ ]:
options(repr.plot.width=12, repr.plot.height=6)
oncoplot(maf = dfx,bgCol = "white",
        top = total_genes,
        
        fontSize = 10,
        SampleNamefontSize = 8,
        titleFontSize = 12,
        legendFontSize = 14,
        annotationTitleFontSize = 10,
        annotationFontSize = 9,
        
        drawColBar = TRUE,
        drawRowBar = TRUE,
        showTumorSampleBarcodes = F,
        writeMatrix = FALSE)

In [ ]:
out_svg = paste0('../KCPG_Variants/plots/KCP_wo_AR_','TT-',gsub("'","", gsub(' ','_',ttype)),'_oncoplot_02.svg')
paste0("Saving : ", out_svg)
svg(out_svg, height=7, width=9)

oncoplot(maf = dfx,bgCol = "white",
        top = total_genes,
        
        fontSize = 10,
        SampleNamefontSize = 8,
        titleFontSize = 12,
        legendFontSize = 14,
        annotationTitleFontSize = 10,
        annotationFontSize = 9,
        
        drawColBar = TRUE,
        drawRowBar = TRUE,
        showTumorSampleBarcodes = F,
        writeMatrix = FALSE)
dev.off()

### Plot Transition and Transversions
returns a list of summarized tables in various ways. Summarized data can also be visualized as a boxplot showing overall distribution of six different conversions and as a stacked barplot showing fraction of conversions in each sample.

In [ ]:
dfx

In [ ]:
options(repr.plot.width=13,repr.plot.height=6)
dfx.titv = titv(maf = dfx, plot = F, useSyn = T  )
plotTiTv(res = dfx.titv ,showBarcodes = F, textSize=5)

In [ ]:
options(repr.plot.width=8, repr.plot.height=8)
set.seed(42)
geneCloud(input = dfx, min=0)

In [ ]:
out_svg = paste0('../KCPG_Variants/plots/KCP_wo_AR_','TT-',gsub("'","", gsub(' ','_',ttype)),'_genecloud_01.svg')
paste0("Saving : ", out_svg)
svg(out_svg, height=8, width=8)
set.seed(42)
geneCloud(input = dfx, min=0)
dev.off()

In [ ]:
options(repr.plot.width=10, repr.plot.height=10)
gs = getGeneSummary(dfx)
gs = gs[MutatedSamples >=0]
set.seed(42)
wordcloud::wordcloud(words = gs[, Hugo_Symbol], gs[,MutatedSamples], 
           min.freq =1,
           max.words=50, 
          random.order=FALSE, 
          rot.per=0.35, 
          colors=brewer.pal(8, "Dark2"))

In [ ]:
options(repr.plot.width=10, repr.plot.height=10)

out_svg = paste0('../KCPG_Variants/plots/KCP_wo_AR_','TT-',gsub("'","", gsub(' ','_',ttype)),'_genecloud_02.svg')
paste0("Saving : ", out_svg)
svg(out_svg, height=9, width=12)
gs = getGeneSummary(dfx)
gs = gs[MutatedSamples >=0]
set.seed(42)
wordcloud::wordcloud(words = gs[, Hugo_Symbol], gs[,MutatedSamples], 
          min.freq = 1,
          # max.words=200, 
          random.order=FALSE, 
          rot.per=0.45, 
          colors=brewer.pal(8, "Dark2"))
dev.off()

##  Drug Interactions
- plot shows potential druggable gene categories along with upto top 5 genes involved in them. 
- One can also extract information on drug-gene interactions. 
- [check this documentation](https://bioconductor.org/packages/release/bioc/vignettes/maftools/inst/doc/maftools.html#98_drug-gene_interactions)

In [ ]:
options(repr.plot.height=6, repr.plot.width=12)
dfx_drug =maftools::drugInteractions(dfx, top=gf_sx )

In [ ]:
out_svg = paste0('../KCPG_Variants/plots/KCP_wo_AR_','TT-',gsub("'","", gsub(' ','_',ttype)),'_drugInteractions_01.svg')
paste0("Saving drug interaction svg to : ", out_svg)
svg(out_svg, height=9, width=12)
dfx_drug =maftools::drugInteractions(dfx, top=gf_sx)
dev.off()

## Oncogenic Pathways

In [ ]:
options(repr.plot.width=6, repr.plot.height=4)
op = OncogenicPathways(dfx)

In [ ]:
op

In [ ]:
out_svg = paste0('../KCPG_Variants/plots/KCP_wo_AR_','TT-',gsub("'","", gsub(' ','_',ttype)),'_OncogenicPathway_01.svg')
paste0("Saving drug interaction svg to : ", out_svg)

svg(out_svg, height=9, width=12)
op
dev.off()

In [ ]:
options(repr.plot.width=10, repr.plot.height=2)
for( i in 1:length( op$data$Pathway ) ){
    pway = op$data$Pathway[i]
    PlotOncogenicPathways(dfx,pway, fullPathway = F, removeNonMutated = T)
}

In [ ]:
# pathway_lst = c("Cell_Cycle","Hippo","MYC","NOTCH","NRF2","PI3K","RTK-RAS","TGF-Beta","TP53","WNT")

In [ ]:
# options(repr.plot.width=10, repr.plot.height=8)
# PlotOncogenicPathways(dfx,pathway_lst[10], fullPathway = T)

In [ ]:
# options(repr.plot.width=10, repr.plot.height=3)
# PlotOncogenicPathways(dfx,pathway_lst[9], fullPathway = T)

In [ ]:
# options(repr.plot.width=10, repr.plot.height=9)
# PlotOncogenicPathways(dfx,pathway_lst[7], fullPathway = T)

In [ ]:
# options(repr.plot.width=10, repr.plot.height=3)
# PlotOncogenicPathways(dfx,pathway_lst[7], fullPathway = F, removeNonMutated = T)

## Save the MAF file

In [ ]:
out_file = paste0('../KCPG_Variants/maf_res/KCP_wo_AR_','TT-',gsub("'","", gsub(' ','_',ttype)) )
paste0("Saving MAF file to : ", out_file)
maftools::write.mafSummary(dfx, out_file)

In [ ]:
out_file = paste0('../KCPG_Variants/maf_res/KCP_wo_AR_','TT-',gsub("'","", gsub(' ','_',ttype)),'_drugInteractions.tsv')
paste0("Saving drug Interactions data to : ", out_file)
write.table(dfx_drug, out_file, sep="\t", row.names = F, quote = F)

## Save the data frames

In [ ]:
out_file = paste0('../KCPG_Variants/data/KCP_wo_AR_','TT-',gsub("'","", gsub(' ','_',ttype)),"_mutation_data.tsv")
paste0("Saving mutation data to : ", out_file)
write.table(df_ss, file = out_file, sep="\t", quote = F, row.names = F)

In [ ]:
out_file = paste0('../KCPG_Variants/data/KCP_wo_AR_','TT-',gsub("'","", gsub(' ','_',ttype)),"_clinical_data.tsv")
paste0("Saving clinical data to : ", out_file)
write.table(df2_ss, file = out_file, sep="\t", quote = F, row.names = F)

In [ ]:
version

In [ ]:
sessionInfo()